# **2-Laboratory-15-10-2020**

| Credits to the authors of the exercises: Andrea Pasini, Giuseppe Attanasio, Flavio Giobergia <br />
| Master of Science in Data Science and Engineering, Politecnico di Torino, A.A. 2020-21

## Global Land Temperature - Data preprocessing
The Global Land Temperature (GLT) dataset is a large collection of measurements actively maintained by Berkeley Earth. It contains the raw source data measured with stations all around the globe, plus anintermediate format and several formatted output files. Data span from ∼1750 up to recent days with monthly and daily availability. 
<br />
Measurements are provided by hemispheres, states, countries, cities andmore. You can read more about the dataset at the Berkeley Earth website. For the purpose of this laboratory you will work on a modified, smaller but dirtier, version of the original GLT dataset, to stress the importance of data preprocessing. More specifically, this didactic version contains the formatted output files of the major cities of the globe with monthly granularity. For the sake of simplicity, the analysis will range between almost two centuries (i.e. between the years 1817 and 2012). The dataset is composed of∼200k rows corresponding to the measurements taken the first day of themonth in a given city. Each measurement is then described by 7 values:
- Date, when the measurement was taken
- AverageTemperature
- AverageTemperatureUncertainty
- City, from which the measurement was taken•
- Country
- Latitude
- Longitude

The main goal of this exercise is to learn how to clean a real-world dataset searching for anomalies, suchas missing values or outliers, in its data

### Questions
1. Load the Global Land Temperature dataset as a list of lists. Before starting, take a moment to better inspect the attributes you are going to work on. How many of them are nominal, how many continuous or discrete?

In [1]:
import csv

# initialize
GLT_dataset = [ [] for i in range(7)]

with open('../Datasets/GLT_filtered.csv') as f:
    
    for row in csv.reader(f): 
        for i in range(len(row)):
            GLT_dataset[i].append(row[i])
        
head = [ GLT_dataset[i].pop(0) for i in range(len(GLT_dataset))]

Let's see better what we obtained

In [2]:
print(f" *** Head  \n{head}")
print("\n *** First 5 elements for each list")

for i in range(len(GLT_dataset)):
    print(GLT_dataset[i][:8])

 *** Head  
['Date', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City', 'Country', 'Latitude', 'Longitude']

 *** First 5 elements for each list
['1849-01-01', '1849-02-01', '1849-03-01', '1849-04-01', '1849-05-01', '1849-06-01', '1849-07-01', '1849-08-01']
['26.704', '27.434', '', '26.14', '25.427', '24.844', '24.058000000000003', '23.576']
['1.435', '1.3619999999999999', '', '1.3869999999999998', '1.2', '1.402', '1.254', '1.265']
['Abidjan', 'Abidjan', 'Abidjan', 'Abidjan', 'Abidjan', 'Abidjan', 'Abidjan', 'Abidjan']
["CÃ´te D'Ivoire", "CÃ´te D'Ivoire", "CÃ´te D'Ivoire", "CÃ´te D'Ivoire", "CÃ´te D'Ivoire", "CÃ´te D'Ivoire", "CÃ´te D'Ivoire", "CÃ´te D'Ivoire"]
['5.63N', '5.63N', '5.63N', '5.63N', '5.63N', '5.63N', '5.63N', '5.63N']
['3.23W', '3.23W', '3.23W', '3.23W', '3.23W', '3.23W', '3.23W', '3.23W']


So, we're dealing with 2 nominal attributes (City,Country), three discrete ones (Date,Latitude,Longitude) and finally two continuos attributes (AverageTemperature,AverageTemperatureUncertainty) <br />

2. Analyze the attribute AverageTemperature, which contains missing values. Fill any gap with the arithmetic mean among the closest antecedent and the closest successive measurements in time,taken in the same city. Assume the following rules for edge cases:

original_list = ['', 5, 6,'']
step_1        = [ 2.5, 5, 6,''] # (0 + 5) / 2
step_2        = [ 2.5, 5, 6,  3 ] # (6 + 0) / 2

original_list   = ['','', 24, 28.9 ]
step_1          = [ 12,'', 24, 28.9 ] # (0 + 24) / 2
step_2          = [ 12, 18, 24, 28.9 ] # (12 + 24) / 2

In [93]:
# let's start with a toy vector in order to see better what happens
toy =  ['',5,4,'',3,'','']

def next_non_negative(i):
    for k in range(i+1,len(toy)):
        if toy[k] != '':
            return k
    return -1

""" 
    I define the first control outside the loop 
    because in this way this control will take place
    just once
"""
print(toy)

if toy[0] == '':
    toy[0] = toy[next_non_negative(0)]/2

print(toy)

# from second to penultimate
for i in range(1,len(toy)):
    
    if toy[i] == '':
        j = next_non_negative(i)
        
        # if there is any non-null value beyond it, it takes the previous one
        if j == -1:
            toy[i] = toy[i-1]/2
        else:
            if (j-i) > 1:
                toy[i] = toy[j]/2
            else:
                toy[i] = (toy[i-1] + toy[j])/2
    print(toy)
    

['', 5, 4, '', 3, '', '']
[2.5, 5, 4, '', 3, '', '']
[2.5, 5, 4, '', 3, '', '']
[2.5, 5, 4, '', 3, '', '']
[2.5, 5, 4, 3.5, 3, '', '']
[2.5, 5, 4, 3.5, 3, '', '']
[2.5, 5, 4, 3.5, 3, 1.5, '']
[2.5, 5, 4, 3.5, 3, 1.5, 0.75]


It'd works, now let's try it on our list, but firstly we need to map these values as float, otherwise we cannot calculate the mean 


In [3]:
# convert from string to float
GLT_dataset[1] = [float(i) if i != '' else '' for i in GLT_dataset[1] ]

def next_non_negativeV2(i):
    for k in range(i+1,len(GLT_dataset[1])):
        if GLT_dataset[1][k] != '':
            return k
    return -1

if GLT_dataset[1][0] == '':
    GLT_dataset[1][0] = GLT_dataset[1][next_non_negative(0)]/2

for i in range(len(GLT_dataset[1])):
    
    if GLT_dataset[1][i] == '':
        
        j = next_non_negativeV2(i)
        
        if j == -1:
            GLT_dataset[1][i] = GLT_dataset[1][i-1]/2
        else:
            if (j-i) > 1:
                GLT_dataset[1][i] = GLT_dataset[1][j]/2
            else:
                GLT_dataset[1][i] = (GLT_dataset[1][i-1] + GLT_dataset[1][j])/2
    
GLT_dataset[1][:10]

[26.704,
 27.434,
 26.787,
 26.14,
 25.427,
 24.844,
 24.058000000000003,
 23.576,
 24.4195,
 25.263]

In [64]:
for i,elem in enumerate(GLT_dataset[1]):
    
    if elem == '' and i != len(GLT_dataset[1])-1:
        # look for the first non-null value
        for j in range(i+1,len(GLT_dataset[1])):
            if GLT_dataset[1][j] != '':
                GLT_dataset[1][i] = GLT_dataset[1][j]/2
                break
                
    elif elem == '' and i == len(GLT_dataset[1])-1:
        # use the previous element
        GLT_dataset[1][i] = GLT_dataset[1][i-1]/2 

GLT_dataset[1][:10]

[26.704,
 27.434,
 13.07,
 26.14,
 25.427,
 24.844,
 24.058000000000003,
 23.576,
 12.6315,
 25.263]

3. Define a function that, given the name of a city and an integerN >0, prints:<br />
(a) the top N hottest measurements; <br />
(b) the top N coldest measurements.

In [39]:
def getTopTemperatures(city,N):
    # array with all the temperatures of records with the given city
    temp_city = [ GLT_dataset[1][i] for i,elem in enumerate(GLT_dataset[3]) if elem == city]
    temp_city.sort()
    
    #      coldest         hottest
    return [temp_city[:N],temp_city[:-N:-1]]

coldest, hottest = getTopTemperatures("Abidjan",5)

print(f"Hottest *** \n{hottest}\n")
print(f"Coldest *** \n{coldest}")

Hottest *** 
[29.923, 29.596, 29.477, 29.471]

Coldest *** 
[11.409500000000003, 12.001, 12.163499999999999, 12.163499999999999, 12.2215]


(*) Let’s search for other anomalies in data distribution with the help of matplotlib. Plot the distribution of the average land temperatures for Rome and Bangkok using the aforementioned histogram plotting function

(*) One might think that Bangkok sensor provide temperature samples in degrees Fahrenheit while the ones located in Rome use the Celsius notation, which is the common representation in the whole dataset. Write a function to transform Fahrenheit measurements back to Celsius, apply it to your data and plot the two distribution again.

## IMDb reviews - Textual data preparation 
This exercise is meant to get you acquainted with the preprocessing of textual data. You can find useful information about Information Retrieval theory in the book "Introduction to Information Retrieval" (Manning, Raghavan, and Schütze 2008) also available at Introduction to Information Retrieval Stanford webpage. <br />

Internet Movie Database (IMDb) is a popular online platform that gathers many information related tomultimed like movies, tv shows, video games and many more. You can navigate to its website to explore the huge amount of updated content it offers. The only difference with the previously used CSV datasetsis that, in this one, the first row of the file contains a “header” (i.e. the name of each column). You should skip the first row and start reading from the second one on. <br />
IMDb has soon become a rich data source for the scientific community. Among the others, people’s reviews are one of the most important types of data that can be retrieved from the platform. The collection of reviews on a specific movie, for example, contains intrinsic information about its approval rating. Therefore, a considerable amount of recent science works has addressed the identification of the sentimentwithin textual reviews and surveys. In the context of movie reviews, the sentiment analysis would seek todiscover if the reviewer liked the movie or not based on the content of the text, with the sentiment being represented as numerical value (e.g. a score between 1 and 10) or a binary one (e.gPositiveorNegative). Wikipedia provides a general overview on the topic of sentiment analysis.During this laboratory you will work on the dataset collected and used by Maas et al. 2011. We will focus on a smaller portion of the whole dataset, known as the training data (you will learn more ontraining and test datasets). It contains 25,000 user reviews collected from IMDb for different movies.Since each review has also a numerical score between 1 and 10, the authors considered the ones witha score lower than 4 to have a negative sentiment, while the ones with a score higher than 6 to have apositive sentiment. The dataset includes12,500positive and12,500negative reviews. For the purpose ofthis laboratory, reviews, which originally came in different files, have been organized in a single CSV file.Each line of the file refers to a single review and has two fields:
1. the textual comment
2. a binary value indicating either the positive or negative sentiment, represented respectively by a ’1’and a ’0’.

### Questions
1. Load the IMDb dataset as a list of lists.

All the rows have the same structure, the comment starts and ends with a ", and it is followed by the relative number. So we can use that structure for the extraction. 

In [106]:
# Using readlines() 
IMDb_dataset = [ [], [] ]
head = ["review","label"]

with open('../Datasets/aclimdb_reviews_train.txt',encoding="utf8") as f:
    # skip 1st line (head)
    next(f)
    
    for line in f:
        """
            the rows have always the same structure
            " -- comment -- ",0/1 \n
            with [1:-3] I get just the comment, without the ""
            with [-1] I get just the last element (number)
        """
        IMDb_dataset[0].append((line.rstrip('\n')[1:-3]))
        IMDb_dataset[1].append((line.rstrip('\n')[-1]))

print(IMDb_dataset[0][:3],"\n\n",IMDb_dataset[1][:3])

['For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan ""The Skipper"" Hale jr. as a police Sgt.', "Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.<br /><br />Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form.", 'Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV\'s ""Flamingo Road"") as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the Gateway to Hell! The scenes with Raines modeling are very well captured, the mood music is perfect, Deborah Raffin is charming as Cristina\'s pal, but when Raines moves into a creepy Brooklyn Heights brownstone (inhabite

2. Apply the tokenization function listed below to your reviews. Please refer to the function’s docstring 1 for the input and output parameters. The tokenization procedure splits each comment in tokens (i.e.separate words).

In [125]:
import string

# written by the instructors
def tokenize(docs):
    """
        Compute the tokens for each document.
        Input: a list of strings. Each item is a document to tokenize.
        Output: a list of lists. Each item is a list containing the tokens of therelative document.
    """
    
    tokens = []
    for doc in docs:
        for punct in string.punctuation:
            doc = doc.replace(punct, " ")
        
        split_doc = [ token.lower() for token in doc.split(" ") if token ]
        tokens.append(split_doc)
    return tokens

In [133]:
tokenList = tokenize(IMDb_dataset[0])

for i in tokenList[:1]:
    print(i, end=" ")

['for', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', 'imagine', 'a', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny', 'maureen', 'stapleton', 'is', 'a', 'scene', 'stealer', 'the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream', 'watch', 'for', 'alan', 'the', 'skipper', 'hale', 'jr', 'as', 'a', 'police', 'sgt'] 

3. The next step requires the computation of the term frequency(TF) of each token within its respective document. Although there exist different techniques to evaluate the frequency, we will now assumethat the TF of a tokentin a documentdis equal to the number of occurrences oftind. Computethe TF for all your reviews.